In [2]:
import torchvision
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
train_path = '../data/train/'
test_path = '../data/test/'

In [4]:
# Resize because they have different sizes, flip because the apples will most likely lay down and not stand up

train_transforms = transforms.Compose([
    transforms.CenterCrop(128),
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
    ])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [5]:
train_dataset = ImageFolder(root=train_path, transform=train_transforms)
test_dataset = ImageFolder(root=test_path, transform=test_transforms)

print(len(train_dataset))

# Access a specific image and its corresponding label
image, label = train_dataset[300]

# Now, image is a tensor
print(type(image))
print(type(label))

print(image.shape)
# print(image)
print(label)

388
<class 'torch.Tensor'>
<class 'int'>
torch.Size([3, 128, 128])
2


In [6]:
class_names = train_dataset.classes

for idx, class_name in enumerate(class_names):
    print(f"Class {idx}: {class_name}")

Class 0: blotch
Class 1: normal
Class 2: rot
Class 3: scap


In [7]:
train_dataset[7]

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 0)

In [8]:
show_imgs(train_dataset)

NameError: name 'show_imgs' is not defined

In [ ]:
print(train_dataset)

NameError: name 'train_dataset' is not defined

batch sizes with the power of 2 starting from 16 until 1024. This is because the batch size needs to fit the memory requirements of the GPU and the architecture of the CPU.

So, the acceptable values for the batch size are 16, 32, 64, 128, 256, 512 and 1024!

In [134]:
# Minibatches because small dataset
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle=False)

In [135]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [136]:
def train(model, train_loader, optimizer, n_epochs):
    device = set_device()
    best_acc = 0
    
    history_dict = {}
    
    for epoch in range(n_epochs):
        print("Epoch number %d " % (epoch + 1) )
        
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for batch in train_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            loss = nn.CrossEntropyLoss(outputs, labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
            
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct/total
        
        history_dict = {}
        print(" - Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f"
              % (running_correct, total, epoch_acc, epoch_loss))
        
    print("Finished training")
    
            
        
        
        

In [137]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    predicted_correct = 0
    total = 0
    device = set_device()
    
    with torch.no_grad():
        for batch in test_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            output = model(images)           
            
            _, predicted = torch.max(output.data, 1)
            
            loss = nn.CrossEntropyLoss(output, labels)
            
            test_loss += loss.item()
            
            predicted_correct += (predicted == labels). sum().item()
            
            acc = predicted_correct / total * 100
            
            epoch_loss = test_loss/len(test_loader)
            epoch_acc = 100.00 * predicted_correct/total
            
    print(f" - Prediction time..... predicted correct: {predicted_correct}  out of {total}. ({acc}% accuracy))")
    return f"epoch_acc: {epoch_acc}"
    

            


In [133]:
history = evaluate(model_resnet18_untrained, test_loader)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [20]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

model_resnet18_trained = models.resnet18(pretrained=True)
model_resnet18_untrained = models.resnet18(pretrained=False)

num_ftrs = model_resnet18_untrained.fc.in_features
number_of_classes = 4 
model_resnet18_untrained.fc = nn.Linear(num_ftrs, number_of_classes)

loss_fn = nn.CrossEntropyLoss()  # experiment with different loss functions

optimizer = optim.SGD(model_resnet18_untrained.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

n_epochs = 150


c:\Users\31641\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\31641\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\31641\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings

In [140]:
train(model_resnet18_untrained, train_loader, optimizer, n_epochs=15)

Epoch number 1 


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [138]:
train_model(model_resnet18_trained, train_loader, test_loader, loss_fn, optimizer, n_epochs=15)

Epoch number 1 


KeyboardInterrupt: 

In [41]:
history_resnet18_untrained = evaluate_model(model_resnet18_untrained, test_loader)
history_resnet18_trained = evaluate_model(model_resnet18_trained,)

In [85]:
type(train_dataset)

torchvision.datasets.folder.ImageFolder

In [ ]:
# Custom covnet

model_custom = nn.Sequential([
    
])